In [220]:
import os
from os.path import join, dirname, pardir
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Imputer

# models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error

#so pandas doesn't spit out a warning everytime
import warnings
warnings.simplefilter(action='ignore', category=DeprecationWarning) 
warnings.simplefilter(action='ignore', category=FutureWarning)

# Import data

In [221]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

In [222]:
print(train.shape)
print(test.shape)

(1460, 81)
(1459, 80)


In [223]:
#Save the 'Id' column
train_ID = train['Id']
test_ID = test['Id']

#Now drop the  'Id' column since it's unnecessary for  the prediction process
train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)

In [224]:
train.dtypes.unique()

array([dtype('int64'), dtype('O'), dtype('float64')], dtype=object)

This data set has three types of data types, which map to two different types of variables; integers and floats -> Numberical variables, and objects -> categorical variables.

To do:
    - deal with null values
    - convert some numerical features to categorical features
    - normalise skewed numerical features
    - convert categorical features to dummies (one hot encoding)
    - standardise features
    
This todo list was born out of reading the following excellent kernals: 
1. https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard
2. https://www.kaggle.com/juliencs/a-study-on-regression-applied-to-the-ames-dataset
    
Before getting started, lets add both training and test datasets together.

In [225]:
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.SalePrice.values
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['SalePrice'], axis=1, inplace=True)
print("all_data size is : {}".format(all_data.shape))

all_data size is : (2919, 79)


# Nulls

In [226]:
total = all_data.isnull().sum().sort_values(ascending=False)
percent = (all_data.isnull().sum()/all_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(24)

,Total,Percent
PoolQC,2909,0.996574
MiscFeature,2814,0.964029
Alley,2721,0.932169
Fence,2348,0.804385
FireplaceQu,1420,0.486468
LotFrontage,486,0.166495
GarageFinish,159,0.054471
GarageQual,159,0.054471
GarageYrBlt,159,0.054471
GarageCond,159,0.054471


23 features of the total 79 explanatory features are missing more than 1 data point. 

## Lets go through them: 

### PoolQC
Pool quality - 99% of the properties are missing this feature. Makes sense...how many houses actually have pools. I do fell having a pool does give some information about house price, so I'm going to create a new binary feature; has pool?, and drop PoolQC.

In [227]:
all_data['hasPool'] = all_data['PoolQC'].apply(lambda x: 0 if pd.isnull(x) else 1)
all_data = all_data.drop('PoolQC', axis=1)

### MiscFeature
Miscellaneous feature not covered in other categories. 96% of the properties persent in the dataset are missing this feature. Lets look at what feaures are present if something is present: 

In [228]:
all_data.MiscFeature.value_counts()

Shed    95
Gar2     5
Othr     4
TenC     1
Name: MiscFeature, dtype: int64

Hmm...if a house has a tennis court or a 2nd garage, this definitely points to a more expensive house. There's so few of them present in the dataset however, I'm going to drop the feature.

In [229]:
all_data = all_data.drop('MiscFeature', axis=1)

### Alley

Type of alley access to property. 93% of properties have no information here and overall I feel it's a poor indicator of value so I'm going to drop it.

In [230]:
all_data = all_data.drop('Alley', axis=1)

### Fence

Fence quality. 80% of properties do not have any information here. This feature gives a sense of the privacy the property has and this is definitely something people values.

In [231]:
all_data.Fence.value_counts()

MnPrv    329
GdPrv    118
GdWo     112
MnWw      12
Name: Fence, dtype: int64

I'm unsure as to whether there'll be good value in this feauture, but I'll create it anyway.

In [232]:
privacy_map = {'MnPrv':1, 'MnWw':1, 'GdPrv':2, 'GdWo':2, np.nan:0}
all_data['PrivacyRating'] = all_data['Fence'].map(lambda x: privacy_map[x])
all_data = all_data.drop('Fence', axis=1)

In [233]:
all_data['PrivacyRating'].value_counts()

0    2348
1     341
2     230
Name: PrivacyRating, dtype: int64

### FireplaceQu
Fireplace quality - Ok so more than 50% of properties in the dataset have information about fireplaces. To me this means that the location in general is cold for at least some of the year, so having a fireplace would be desireable in the property. The order of each value is important as Ex (Excellent) is far more valuable than Po (Poor), so I'm going to create an ordinal feauture.

In [234]:
all_data.FireplaceQu.value_counts()

Gd    744
TA    592
Fa     74
Po     46
Ex     43
Name: FireplaceQu, dtype: int64

In [235]:
all_data = all_data.replace({"FireplaceQu" : {np.nan: 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5}})

### LotFrontage

Linear feet of street connected to property. I'm honestly not too sure what this feature is telling me. 17% of properties are missing this feature. For now I'm going to drop this feature.

In [236]:
all_data = all_data.drop('LotFrontage', axis=1)

### GarageFinish / GarageQual / GarageYrBlt / GarageCond / GarageType

All these garage features are missing 5% of thier values. I'm going to ordinally encode all of them, except GarageYrBlt which is already fine, and GarageType which just needs to be encoded.

In [237]:
all_data.GarageType.value_counts()

Attchd     1723
Detchd      779
BuiltIn     186
Basment      36
2Types       23
CarPort      15
Name: GarageType, dtype: int64

In [238]:
all_data.loc[:, "GarageType"] = all_data.loc[:, "GarageType"].fillna("No")
all_data.loc[:, "GarageYrBlt"] = all_data.fillna(all_data.loc[:, "GarageYrBlt"].median())

In [239]:
all_data = all_data.replace({"GarageFinish" : {np.nan: 0, "Unf": 1, "RFn": 2, "Fin": 3},
                             "GarageQual" : {np.nan: 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4 , "Ex": 5},
                             "GarageCond" : {np.nan: 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4 , "Ex": 5}
                            })

### BsmtCond / BsmtExposure / BsmtQual

Evaluates the general condition of the basement / Refers to walkout or garden level walls / Evaluates the height of the basement

2% of properties do not have basement information. Again order has value here so I'll encoded them ordinally.

In [240]:
all_data.BsmtQual.value_counts()

TA    1283
Gd    1209
Ex     258
Fa      88
Name: BsmtQual, dtype: int64

In [241]:
all_data = all_data.replace({
    "BsmtCond": {np.nan: 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4 , "Ex": 5},
    "BsmtExposure": {np.nan: 0, "No": 1, "Mn": 2, "Av": 3, "Gd": 4},
    "BsmtQual": {np.nan: 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4 , "Ex": 5}
})

### BsmtFinType1 / BsmtFinType2

Rating of basement finished area / Rating of basement finished area (if multiple types)

In [242]:
all_data.BsmtFinType1.value_counts()

Unf    851
GLQ    849
ALQ    429
Rec    288
BLQ    269
LwQ    154
Name: BsmtFinType1, dtype: int64

In [243]:
all_data.loc[:, "BsmtFinType1"] = all_data.loc[:, "BsmtFinType1"].fillna("No")
all_data.loc[:, "BsmtFinType2"] = all_data.loc[:, "BsmtFinType2"].fillna("No")

In [244]:
all_data = all_data.replace({
    "BsmtFinType1" : {"No" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, "ALQ" : 5, "GLQ" : 6},
    "BsmtFinType2" : {"No" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, "ALQ" : 5, "GLQ" : 6}
})

### MasVnrType / MasVnrArea

Masonry veneer type / Masonry veneer area in square feet

Order does not matter for both of these features, so I'll just fill in the null values and move on.

In [245]:
all_data.MasVnrType.value_counts()

None       1742
BrkFace     879
Stone       249
BrkCmn       25
Name: MasVnrType, dtype: int64

In [246]:
all_data.loc[:, "MasVnrType"] = all_data.loc[:, "MasVnrType"].fillna("None")
all_data.loc[:, "MasVnrArea"] = all_data.loc[:, "MasVnrArea"].fillna(0)

### MSZoning

Identifies the general zoning classification of the sale.

In [247]:
all_data.MSZoning.value_counts()

RL         2265
RM          460
FV          139
RH           26
C (all)      25
Name: MSZoning, dtype: int64

Hmmm....I don't think order is hugely important here. So I'll just encode this features with dummy variables at the very end. 4 rows are null so I'm going to fill them with the most common value: RL.

In [248]:
all_data.loc[:, "MSZoning"] = all_data.loc[:, "MSZoning"].fillna("RL")

### BsmtHalfBath / BsmtFullBath

Basement half bathrooms / Basement full bathrooms

In [249]:
all_data.BsmtHalfBath.value_counts()

0.0    2742
1.0     171
2.0       4
Name: BsmtHalfBath, dtype: int64

In [250]:
all_data.loc[:, "BsmtHalfBath"] = all_data.loc[:, "BsmtHalfBath"].fillna(0)
all_data.loc[:, "BsmtFullBath"] = all_data.loc[:, "BsmtFullBath"].fillna(0)

### Utilities

Type of utilities available

In [251]:
all_data.loc[:, "Utilities"] = all_data.loc[:, "Utilities"].fillna("AllPub")
all_data = all_data.replace({
    "Utilities" : {"ELO" : 1, "NoSeWa" : 2, "NoSewr" : 3, "AllPub" : 4}
})

### Functional

Home functionality (Assume typical unless deductions are warranted)

In [252]:
all_data.Functional.value_counts()

Typ     2717
Min2      70
Min1      65
Mod       35
Maj1      19
Maj2       9
Sev        2
Name: Functional, dtype: int64

In [253]:
all_data.loc[:, "Functional"] = all_data.loc[:, "Functional"].fillna("Typ")
all_data = all_data.replace({
    "Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, "Min2" : 6, "Min1" : 7, "Typ" : 8}
})

Ok most of the features with missing data have been dealt with. Lets recheck the whole dataset for nulls and check for features I've missed...

In [254]:
total = all_data.isnull().sum().sort_values(ascending=False)
percent = (all_data.isnull().sum()/all_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data[missing_data.Total > 0]

,Total,Percent
SaleType,1,0.000343
GarageArea,1,0.000343
Exterior1st,1,0.000343
Exterior2nd,1,0.000343
BsmtUnfSF,1,0.000343
BsmtFinSF2,1,0.000343
BsmtFinSF1,1,0.000343
Electrical,1,0.000343
TotalBsmtSF,1,0.000343
KitchenQual,1,0.000343


Ok lets quickly impute the final few missing values.

In [255]:
# Filling with the most common variable
all_data.loc[:, "SaleType"] = all_data.loc[:, "SaleType"].fillna("WD")

# I checked this item line item and it does have a garage, so filling in the most common value (0), would be incorrect.
# I will impute the value as the overall median instead
all_data.loc[:, "GarageArea"] = all_data.loc[:, "GarageArea"].fillna(all_data.loc[:, "GarageArea"].median())

# Makes most sense to impute the most common value here
all_data.loc[:, "Electrical"] = all_data.loc[:, "Electrical"].fillna("SBrkr")

# BsmtFinSF2 - Type 2 finished square feet - 0 is by far the most common value here so I'm going to impute this value
all_data.loc[:, "BsmtFinSF2"] = all_data.loc[:, "BsmtFinSF2"].fillna(0)

# Exterior1st / Exterior2nd - Imputing the most common value
all_data.loc[:, "Exterior1st"] = all_data.loc[:, "Exterior1st"].fillna("VinylSd")
all_data.loc[:, "Exterior2nd"] = all_data.loc[:, "Exterior2nd"].fillna("VinylSd")

# BsmtUnfSF - We don't know if this property has a basement or not, so lets impute it as 0.
all_data.loc[:, "BsmtUnfSF"] = all_data.loc[:, "BsmtUnfSF"].fillna(0)

# KitchenQual - Imputing the most common value
all_data.loc[:, "KitchenQual"] = all_data.loc[:, "KitchenQual"].fillna("TA")

# GarageCars - This property has a garage so I will impute the most common value, 2
all_data.loc[:, "GarageCars"] = all_data.loc[:, "GarageCars"].fillna(2.0)

# TotalBsmtSF - We don't know if this property has a basement or not, so lets impute it as 0.
all_data.loc[:, "TotalBsmtSF"] = all_data.loc[:, "TotalBsmtSF"].fillna(0)

# BsmtFinSF1 - We don't know if this property has a basement or not, so lets impute it as 0.
all_data.loc[:, "BsmtFinSF1"] = all_data.loc[:, "BsmtFinSF1"].fillna(0)

## Boom! All the missing data is now dealt with!

In [256]:
all_data.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold,hasPool,PrivacyRating
0,856,854,0,3,1Fam,3,1,706.0,0.0,6,...,Pave,8,856.0,4,0,2003,2003,2008,0,0
1,1262,0,0,3,1Fam,3,4,978.0,0.0,5,...,Pave,6,1262.0,4,298,1976,1976,2007,0,0
2,920,866,0,3,1Fam,3,2,486.0,0.0,6,...,Pave,6,920.0,4,0,2001,2002,2008,0,0
3,961,756,0,3,1Fam,4,1,216.0,0.0,5,...,Pave,7,756.0,4,0,1915,1970,2006,0,0
4,1145,1053,0,4,1Fam,3,3,655.0,0.0,6,...,Pave,9,1145.0,4,192,2000,2000,2008,0,0


In [257]:
categorical_features = all_data.select_dtypes(include = ["object"]).columns
numerical_features = all_data.select_dtypes(exclude = ["object"]).columns

In [258]:
print('There are {} categorical features'.format(len(categorical_features)))
print('There are {} numerical features'.format(len(numerical_features)))

There are 28 categorical features
There are 48 numerical features


In [264]:
all_data.loc[:, categorical_features].head()

,BldgType,CentralAir,Condition1,Condition2,Electrical,ExterCond,ExterQual,Exterior1st,Exterior2nd,Foundation,...,LotShape,MSZoning,MasVnrType,Neighborhood,PavedDrive,RoofMatl,RoofStyle,SaleCondition,SaleType,Street
0,1Fam,Y,Norm,Norm,SBrkr,TA,Gd,VinylSd,VinylSd,PConc,...,Reg,RL,BrkFace,CollgCr,Y,CompShg,Gable,Normal,WD,Pave
1,1Fam,Y,Feedr,Norm,SBrkr,TA,TA,MetalSd,MetalSd,CBlock,...,Reg,RL,None,Veenker,Y,CompShg,Gable,Normal,WD,Pave
2,1Fam,Y,Norm,Norm,SBrkr,TA,Gd,VinylSd,VinylSd,PConc,...,IR1,RL,BrkFace,CollgCr,Y,CompShg,Gable,Normal,WD,Pave
3,1Fam,Y,Norm,Norm,SBrkr,TA,TA,Wd Sdng,Wd Shng,BrkTil,...,IR1,RL,None,Crawfor,Y,CompShg,Gable,Abnorml,WD,Pave
4,1Fam,Y,Norm,Norm,SBrkr,TA,Gd,VinylSd,VinylSd,PConc,...,IR1,RL,BrkFace,NoRidge,Y,CompShg,Gable,Normal,WD,Pave


First, lets transform some of the numericals, that are in fact categorical, into such.

In [265]:
# MSSubClass=The building class
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)

# Changing OverallCond into a categorical variable
all_data['OverallCond'] = all_data['OverallCond'].astype(str)

# Year and month sold are transformed into categorical features.
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

# Time to deal with numerical skewness in the data

# Final few bits

In [269]:
## Encode all categorical data
all_data = pd.get_dummies(all_data)

In [271]:
train = all_data[:ntrain]
test = all_data[ntrain:]

# Predictions

In [276]:
# RandomForest
regr = RandomForestRegressor()

In [275]:
regr.fit(train, y_train)
predictions = regr.predict(test)

In [278]:
my_submission = pd.DataFrame({'Id': test_ID, 'SalePrice': predictions})
filename = 'RandomForestRegressor_20180904'
my_submission.to_csv('../submissions/{}.csv'.format(filename), index=False)